# Invoicing

Now that we have set up our user info, clients, contracts and projects, as well as a source for time tracking data, we are ready to automatically generate invoices.

## Preamble

In [1]:
from pathlib import Path
import datetime
from IPython import display

In [2]:
import tuttle

In [3]:
app = tuttle.app.App(verbose=False)

## Workflow

_1. Select a project_

In [4]:
my_project = app.get_project(title="Heating Repair")

/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


2. Select a time tracking data source.

In [5]:
from tuttle.calendar import FileCalendar

timetracking_calendar_path = Path("../../tests/data/TuttleDemo-TimeTracking.ics")
my_calendar = FileCalendar(
    path=timetracking_calendar_path, 
    name="TimeTracking"
)

3. Generate one or more timesheets

In [6]:
my_timesheet = tuttle.timetracking.generate_timesheet(
    source=my_calendar,
    project=my_project,
    period="February 2022",
    item_description=my_project.title,
)

/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:101: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return super().execute(  # type: ignore


4. Generate an invoice for the timesheet(s).

In [7]:
my_invoice = tuttle.invoicing.generate_invoice(
    timesheets=[
        my_timesheet,
    ],
    contract=my_project.contract,
    date=datetime.date.today(),
)

5. Render the invoice to a document template:

In [8]:
invoice_dir = Path.home() / "Downloads"

In [9]:
tuttle.rendering.render_invoice(
    user=app.user, 
    invoice=my_invoice,
    style="anvil",
    out_dir=invoice_dir,
)

In [10]:
invoice_path = str(invoice_dir / f"Invoice-{my_invoice.number}" / f"Invoice-{my_invoice.number}.html")

In [11]:
!qlmanage -p {invoice_path}

Testing Quick Look preview with files:
	/Users/cls/Downloads/Invoice-2022-02-21-01/Invoice-2022-02-21-01.html
2022-02-21 21:55:53.852 qlmanage[11525:332330] *** CFMessagePort: bootstrap_register(): failed 1100 (0x44c) 'Permission denied', port = 0x14003, name = 'com.apple.coredrag'
See /usr/include/servers/bootstrap_defs.h for the error codes.
2022-02-21 21:55:53.883 qlmanage[11525:332330] *** CFMessagePort: bootstrap_register(): failed 1100 (0x44c) 'Permission denied', port = 0xca03, name = 'com.apple.tsm.portname'
See /usr/include/servers/bootstrap_defs.h for the error codes.
2022-02-21 21:55:59.974 qlmanage[11525:332343] Persistent UI failed to open file file:///Users/cls/Library/Saved%20Application%20State/com.apple.quicklook.qlmanage.savedState/window_1.data: No such file or directory (2)
